In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('http://bit.ly/kaggletrain') #import training set
test = pd.read_csv('http://bit.ly/kaggletest') #import testing set

In [2]:
#fill Age nan with median
median = train["Age"].median()
train["Age"].fillna(median, inplace=True) 

#fill fare nan with median
median = train["Fare"].median()
train["Fare"].fillna(median, inplace=True) 

#change sex to numerical
train_sex = train[["Sex"]]
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
train_sex_num = ordinal_encoder.fit_transform(train_sex)
train.Sex = train_sex_num

#change ticket to numerical
train_tick = train[["Ticket"]]
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
train_tick_num = ordinal_encoder.fit_transform(train_tick)
train.Ticket = train_tick_num

train.isnull().sum() # check for nan values

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [3]:
#drop labels
train_d = train.drop("Cabin", axis=1) 
train_d = train_d.drop("Embarked", axis=1) 
train_d = train_d.drop("Name", axis=1) 
train_d = train_d.drop("PassengerId", axis=1) 
train_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null float64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null float64
Fare        891 non-null float64
dtypes: float64(4), int64(4)
memory usage: 55.8 KB


In [4]:
train_d.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
dtype: int64

In [5]:
train_d.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare
0,0,3,1.0,22.0,1,0,523.0,7.2500
1,1,1,0.0,38.0,1,0,596.0,71.2833
2,1,3,0.0,26.0,0,0,669.0,7.9250
3,1,1,0.0,35.0,1,0,49.0,53.1000
4,0,3,1.0,35.0,0,0,472.0,8.0500


In [6]:
train_y = train_d.Survived # set class labels
train_x = train_d.drop("Survived", axis=1) # drop Survived from training features
train_x.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare
0,3,1.0,22.0,1,0,523.0,7.2500
1,1,0.0,38.0,1,0,596.0,71.2833
2,3,0.0,26.0,0,0,669.0,7.9250
3,1,0.0,35.0,1,0,49.0,53.1000
4,3,1.0,35.0,0,0,472.0,8.0500


In [21]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

poly_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=2, coef0=1, C=5))
    ])
poly_kernel_svm_clf.fit(train_x, train_y)
cross_val_score(poly_kernel_svm_clf, train_x, train_y, cv=3, scoring="accuracy")

array([0.8047138 , 0.82828283, 0.81818182])

In [22]:
# set testing data

#fill Age nan with median
median = test["Age"].median()
test["Age"].fillna(median, inplace=True) 

#fill fare nan with median
median = test["Fare"].median()
test["Fare"].fillna(median, inplace=True) 

#change sex to numerical
test_sex = test[["Sex"]]
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
test_sex_num = ordinal_encoder.fit_transform(test_sex)
test.Sex = test_sex_num

#change ticket to numerical
test_tick = test[["Ticket"]]
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
test_tick_num = ordinal_encoder.fit_transform(test_tick)
test.Ticket = test_tick_num

test.isnull().sum() # check for nan values


PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [23]:
#drop labels
test_d = test.drop("Cabin", axis=1) 
test_d = test_d.drop("Embarked", axis=1) 
test_d = test_d.drop("Name", axis=1) 
test_d = test_d.drop("PassengerId", axis=1) 
test_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass    418 non-null int64
Sex       418 non-null float64
Age       418 non-null float64
SibSp     418 non-null int64
Parch     418 non-null int64
Ticket    418 non-null float64
Fare      418 non-null float64
dtypes: float64(4), int64(3)
memory usage: 22.9 KB


In [24]:
test_d.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare
0,3,1.0,34.5,0,0,152.0,7.8292
1,3,0.0,47.0,1,0,221.0,7.0000
2,2,1.0,62.0,0,0,73.0,9.6875
3,3,1.0,27.0,0,0,147.0,8.6625
4,3,0.0,22.0,1,1,138.0,12.2875


In [25]:
# classify test data
test['Survived'] = poly_kernel_svm_clf.predict(test_d)

In [27]:
# save classfication result
pd.DataFrame({'PassengerId':test.PassengerId, 'Survived':test.Survived}).set_index('PassengerId').to_csv('sub.csv')